In [1]:
import json
import math
import os
import time
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler, FileCreatedEvent

In [2]:
base = os.path.join(os.path.sep, 'home')
i_dir = os.path.join(os.path.sep, base, 'vector_speed_forecast_daily')
o_dir = os.path.join(os.path.sep, base, 'leaflet_velocity')
i_file = os.path.join(os.path.sep, i_dir, "vector_speed_forecast_daily_1.nc")

In [3]:
def get_datetime(tdelta):
    return (datetime(1950,1,1) + timedelta(hours=tdelta)).strftime('%Y-%m-%d %H:%M:%S')

In [4]:
def get_data(variable_data, time=0, depth=0):
    v=[]
    for lat in reversed(range(1, variable_data.data.shape[2], 12)):
        for lon in range(0, variable_data.data.shape[3], 12):
            value = variable_data.data[time, depth, lat, lon]
            v.append(round(value, 2) if not xr.ufuncs.isnan(value) else 0.)
    return v

In [5]:
def variable_to_json(file, variable_name, time, pnumber):
    netcdf_data = xr.open_dataset(file)
    variable_data = netcdf_data[variable_name]
    json_data = {}
    json_data['header'] = {}
    json_data['header']['dx'] = 1.
    json_data['header']['dy'] = 1.
    json_data['header']['la1'] = netcdf_data['latitude'].attrs['valid_max']
    json_data['header']['la2'] = netcdf_data['latitude'].attrs['valid_min']
    json_data['header']['lo1'] = netcdf_data['longitude'].attrs['valid_max']
    json_data['header']['lo2'] = netcdf_data['longitude'].attrs['valid_min']
    json_data['header']['nx'] = 360
    json_data['header']['ny'] = 170
    json_data['header']['parameterCategory'] = 2
    json_data['header']['parameterNumber'] = pnumber
    if (variable_name == 'uo'):
        json_data['header']['parameterNumberName'] = 'Eastward current'
    else:
        json_data['header']['parameterNumberName'] = 'Northward current'
    json_data['header']['parameterUnit'] = 'm.s-1'
    json_data['header']['refTime'] = pd.to_datetime(time).strftime('%Y-%m-%d %H:%M:%S')

    json_data['data'] = get_data(variable_data)
    return json_data

In [6]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [7]:
def netcdf_to_json(file):
    data = xr.open_dataset(file)
    for time in data['time'].data:
        output = []
        output.append(variable_to_json(file, 'uo', time, 2))
        output.append(variable_to_json(file, 'vo', time, 3))
        outfile = os.path.join(os.path.sep, o_dir, pd.to_datetime(time).strftime('%Y-%m-%d') + '.json')
        with open(outfile, 'w') as f:
            f.write(json.dumps(output, cls=MyEncoder))

In [8]:
class EventHandler(FileSystemEventHandler):
    def on_created(self, event):
        if isinstance(event, FileCreatedEvent):
            netcdf_to_json(event.src_path)

In [ ]:
event_handler = EventHandler()
observer = Observer()
observer.schedule(event_handler, i_dir, recursive=True)
observer.start()
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()